# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [52]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [53]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)


# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,remedios,22.4947,-79.5458,22.51,82,69,3.45,CU,1682679952
1,1,raghunathpur,23.5500,86.6667,36.19,22,91,5.10,IN,1682679952
2,2,puerto baquerizo moreno,-0.9000,-89.6000,25.88,82,68,4.13,EC,1682679954
3,3,port-aux-francais,-49.3500,70.2167,6.17,70,20,10.15,TF,1682679954
4,4,qaqortoq,60.7167,-46.0333,0.61,61,6,3.87,GL,1682679955


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [54]:
%%capture --no-display

# Configure the map plot
city_humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols = ["Max Temp"]
)

# Display the map plot
city_humidity_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [55]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_city_data_df =city_data_df[(city_data_df["Max Temp"]>21) 
                                    & (city_data_df["Max Temp"]<27)
                                     & (city_data_df["Wind Speed"]<4.5)
                                    & (city_data_df["Cloudiness"]==0)]

# Drop any rows with null values
filtered_city_data_df=filtered_city_data_df.dropna()

# Display sample data
filtered_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
87,87,san patricio,28.0170,-97.5169,23.92,97,0,2.74,US,1682680005
110,110,hawaiian paradise park,19.5933,-154.9731,21.05,94,0,1.54,US,1682680019
357,357,nova xavantina,-14.6667,-52.3500,23.05,85,0,0.53,BR,1682680054
386,386,nova vicosa,-17.8919,-39.3719,25.35,77,0,2.83,BR,1682680198
387,387,puerto maldonado,-12.6000,-69.1833,22.22,94,0,1.03,PE,1682680199
396,396,hilo,19.7297,-155.0900,24.04,88,0,1.54,US,1682680206
505,505,prado,-17.3411,-39.2208,25.28,79,0,2.91,BR,1682680276
514,514,kuqa,41.7278,82.9364,23.01,11,0,1.63,CN,1682680282
532,532,vila velha,-20.3297,-40.2925,25.97,87,0,1.54,BR,1682680278
551,551,saint croix,17.7397,-64.7388,24.98,89,0,2.57,VI,1682680308


### Step 3: Create a new DataFrame called `hotel_df`.

In [56]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=filtered_city_data_df[["City", "Country","Lng","Lat","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=''

# Display sample data
hotel_df

,City,Country,Lng,Lat,Humidity,Hotel Name
87,san patricio,US,-97.5169,28.0170,97,
110,hawaiian paradise park,US,-154.9731,19.5933,94,
357,nova xavantina,BR,-52.3500,-14.6667,85,
386,nova vicosa,BR,-39.3719,-17.8919,77,
387,puerto maldonado,PE,-69.1833,-12.6000,94,
396,hilo,US,-155.0900,19.7297,88,
505,prado,BR,-39.2208,-17.3411,79,
514,kuqa,CN,82.9364,41.7278,11,
532,vila velha,BR,-40.2925,-20.3297,87,
551,saint croix,VI,-64.7388,17.7397,89,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [59]:
# Set parameters to search for a hotel
radius = 10000
limit = 20
categories = "accommodation.hotel"

params = {
    "limit":limit,
    "apiKey":geoapify_key,
    "categories":categories   
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude= hotel_df.loc[index, "Lat"]
    longitude= hotel_df.loc[index, "Lng"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
       
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
san patricio - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
nova xavantina - nearest hotel: Hotel Morada do Sol
nova vicosa - nearest hotel: Pousada Pontal da Barra
puerto maldonado - nearest hotel: Puerto Palmeras
hilo - nearest hotel: Dolphin Bay Hotel
prado - nearest hotel: Pousada Recanto do Prado
kuqa - nearest hotel: Kuqa Hotel - foreigners accepted
vila velha - nearest hotel: Hotel Prainha
saint croix - nearest hotel: The Buccaneer


,City,Country,Lng,Lat,Humidity,Hotel Name
87,san patricio,US,-97.5169,28.0170,97,No hotel found
110,hawaiian paradise park,US,-154.9731,19.5933,94,No hotel found
357,nova xavantina,BR,-52.3500,-14.6667,85,Hotel Morada do Sol
386,nova vicosa,BR,-39.3719,-17.8919,77,Pousada Pontal da Barra
387,puerto maldonado,PE,-69.1833,-12.6000,94,Puerto Palmeras
396,hilo,US,-155.0900,19.7297,88,Dolphin Bay Hotel
505,prado,BR,-39.2208,-17.3411,79,Pousada Recanto do Prado
514,kuqa,CN,82.9364,41.7278,11,Kuqa Hotel - foreigners accepted
532,vila velha,BR,-40.2925,-20.3297,87,Hotel Prainha
551,saint croix,VI,-64.7388,17.7397,89,The Buccaneer


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [63]:
%%capture --no-display

# Configure the map plot
hotel_df_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map plot
hotel_df_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)